# TCGA-UCEC — Notebook 05
## Tâche supervisée clinique et baseline MLP

Objectif : établir une baseline supervisée à l’aide d’un MLP simple
en prédisant une variable clinique valide à partir des données RNA-seq.

In [1]:
# ==========================================================================================================
import os                                       # Navigation fichiers (DIRS, chemins relatifs)
import warnings                                 # Masquer warnings (dépréciation)
import gc                                       # Gestion mémoire (nettoyage objets inutilisés)
import json                                     # Lecture du dictionnaire de métadonnées

import numpy as np
import matplotlib.pyplot as plt

import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import roc_auc_score, f1_score, confusion_matrix

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping

# ==========================================================================================================
PROJECT_ROOT = r"C:\Z\M2_AIDA\TCGA_UCEC_project" #Laïla

DIRS = {
    # Racine data
    "DATA": os.path.join(PROJECT_ROOT, "data"),

    # Données
    "RAW": os.path.join(PROJECT_ROOT, "data", "raw"),

    # Données intermédiaires (persistées)
    "PROCESSED":        os.path.join(PROJECT_ROOT, "data", "processed"),
    "NORM":             os.path.join(PROJECT_ROOT, "data", "processed", "normalized"),
    "QC_FILTERED":      os.path.join(PROJECT_ROOT, "data", "processed", "qc_filtered"),
    "COHORT_FILTERED":  os.path.join(PROJECT_ROOT, "data", "processed", "cohort_filtered"),
    "SC_OBJECTS":       os.path.join(PROJECT_ROOT, "data", "processed", "single_cell_objects"),
    "PSEUDOBULK_PROC":  os.path.join(PROJECT_ROOT, "data", "processed", "pseudobulk"),

    # Artefacts (par étape du pipeline)
    "ARTEFACTS": os.path.join(PROJECT_ROOT, "data", "artefacts"),

    "EDA":         os.path.join(PROJECT_ROOT, "data", "artefacts", "exploratory_data_analysis"),
    "QC":          os.path.join(PROJECT_ROOT, "data", "artefacts", "qc_analysis"),
    "COHORT":      os.path.join(PROJECT_ROOT, "data", "artefacts", "cohort_selection"),
    "SINGLE_CELL": os.path.join(PROJECT_ROOT, "data", "artefacts", "single_cell_analysis"),
    "PSEUDOBULK":  os.path.join(PROJECT_ROOT, "data", "artefacts", "pseudobulk_preparation"),
    "DE":          os.path.join(PROJECT_ROOT, "data", "artefacts", "differential_expression"),
    "ENRICH":      os.path.join(PROJECT_ROOT, "data", "artefacts", "functional_enrichment"),

    # Autres dossiers du projet
    "RESULTS_R": os.path.join(PROJECT_ROOT, "Results_R_Analysis"),
    "TMP":       os.path.join(PROJECT_ROOT, "tmp_cache"),
    "DOCS":      os.path.join(PROJECT_ROOT, "documentation"),
}
for path in DIRS.values():
    os.makedirs(path, exist_ok=True)
os.chdir(PROJECT_ROOT)

# ==========================================================================================================
warnings.filterwarnings("ignore") 
plt.rcParams['figure.dpi'] = 100
plt.rcParams['savefig.dpi'] = 300
plt.rcParams['figure.figsize'] = (10, 6) 

# ==========================================================================================================
print(f"✅ Environnement chargé. Working directory: {os.getcwd()}")


✅ Environnement chargé. Working directory: c:\Z\M2_AIDA\TCGA_UCEC_project


## 1. Chargement des données patient-level

Les données chargées ici correspondent aux sorties du Notebook 04

In [2]:
expr_filename = "expr_patient_level_tcga_ucec.tsv"
clin_filename = "clin_patient_level_tcga_ucec.tsv"

expr_path = os.path.join(DIRS["PROCESSED"], expr_filename)
clin_path = os.path.join(DIRS["PROCESSED"], clin_filename)

expr = pd.read_csv(expr_path, sep="\t", index_col=0)
clin = pd.read_csv(clin_path, sep="\t", index_col=0)

print(
    f"✅ Donnees chargees\n"
    f"   - Expression : {expr.shape} (genes x patients)\n"
    f"   - Clinique   : {clin.shape} (patients x variables)"
)


✅ Donnees chargees
   - Expression : (31876, 557) (genes x patients)
   - Clinique   : (597, 78) (patients x variables)


In [3]:
dup_expr = expr.columns[expr.columns.duplicated()]
print("Nombre de colonnes dupliquées (expr):", len(dup_expr))
print("Exemples de patients dupliqués (expr):")
print(dup_expr[:10])


Nombre de colonnes dupliquées (expr): 0
Exemples de patients dupliqués (expr):
Index([], dtype='object')


In [4]:
expr_counts = expr.columns.value_counts()

print("Distribution du nombre de samples par patient (expr):")
print(expr_counts.value_counts().sort_index())


Distribution du nombre de samples par patient (expr):
count
1    557
Name: count, dtype: int64


## 2. Définition de la tâche supervisée

Les sous-types moléculaires TCGA (POLE / MSI / CN-low / CN-high)
ne sont pas directement disponibles dans les données cliniques Xena
pour la cohorte UCEC.

Nous définissons donc une tâche supervisée alternative, biologiquement
pertinente et reproductible : **la prédiction du grade tumoral**
(`tumor_grade.diagnoses`) à partir des profils d'expression RNA-seq.

Ce choix est courant dans les analyses transcriptomiques exploratoires
et permet d'évaluer la capacité du modèle à capturer un signal clinique
à partir des données moléculaires.

In [5]:
print(globals().keys())


dict_keys(['__name__', '__doc__', '__package__', '__loader__', '__spec__', '__builtin__', '__builtins__', '_ih', '_oh', '_dh', 'In', 'Out', 'get_ipython', 'exit', 'quit', 'open', '_', '__', '___', '__vsc_ipynb_file__', '_i', '_ii', '_iii', '_i1', 'os', 'warnings', 'gc', 'json', 'np', 'plt', 'pd', 'train_test_split', 'LabelEncoder', 'roc_auc_score', 'f1_score', 'confusion_matrix', 'tf', 'Sequential', 'Dense', 'Dropout', 'EarlyStopping', 'PROJECT_ROOT', 'DIRS', 'path', '_i2', 'expr_filename', 'clin_filename', 'expr_path', 'clin_path', 'expr', 'clin', '_i3', 'dup_expr', '_i4', 'expr_counts', '_i5'])


In [6]:
print("tumor_grade.diagnoses in columns:",
      "tumor_grade.diagnoses" in clin.columns)

print("\nValeurs brutes :")
print(clin["tumor_grade.diagnoses"].head(20))

print("\nDistribution :")
print(clin["tumor_grade.diagnoses"].value_counts(dropna=False))


tumor_grade.diagnoses in columns: True

Valeurs brutes :
TCGA-2E-A9G8    Not Reported
TCGA-4E-A92E    Not Reported
TCGA-5B-A90C    Not Reported
TCGA-5S-A9Q8    Not Reported
TCGA-A5-A0G1    Not Reported
TCGA-A5-A0G2    Not Reported
TCGA-A5-A0G3    Not Reported
TCGA-A5-A0G5    Not Reported
TCGA-A5-A0G9    Not Reported
TCGA-A5-A0GA    Not Reported
TCGA-A5-A0GB    Not Reported
TCGA-A5-A0GD    Not Reported
TCGA-A5-A0GE    Not Reported
TCGA-A5-A0GG    Not Reported
TCGA-A5-A0GH    Not Reported
TCGA-A5-A0GI    Not Reported
TCGA-A5-A0GJ    Not Reported
TCGA-A5-A0GM    Not Reported
TCGA-A5-A0GN    Not Reported
TCGA-A5-A0GP    Not Reported
Name: tumor_grade.diagnoses, dtype: object

Distribution :
tumor_grade.diagnoses
Not Reported    585
NaN              12
Name: count, dtype: int64


In [7]:
clin.nunique().sort_values(ascending=False).head(20)


sample                                                       597
sample_id.samples                                            597
pathology_report_uuid.samples                                558
case_id                                                      557
id                                                           557
updated_datetime.treatments.diagnoses                        545
created_datetime.treatments.diagnoses                        545
submitter_id.treatments.diagnoses                            545
treatment_id.treatments.diagnoses                            545
age_at_diagnosis.diagnoses                                   533
age_at_earliest_diagnosis_in_years.diagnoses.xena_derived    533
age_at_earliest_diagnosis.diagnoses.xena_derived             533
days_to_birth.demographic                                    533
days_to_last_follow_up.diagnoses                             453
days_to_collection.samples                                   448
initial_weight.samples   

In [8]:
from pathlib import Path
import re

# adapte si ton project root est ailleurs
ROOT = Path(".").resolve()

patterns = [
    re.compile(r"subtype|subtypes|molecular|po le|pole|msi|cn[-_ ]?low|cn[-_ ]?high|copy|serous|endometr", re.I),
    re.compile(r"ucec|uter", re.I),
]

candidates = []
for p in ROOT.rglob("*"):
    if p.is_file() and p.suffix.lower() in [".tsv", ".csv", ".txt", ".xlsx", ".json"]:
        name = p.name
        if patterns[0].search(name) or patterns[1].search(name):
            candidates.append(p)

print("Nb fichiers candidats:", len(candidates))
for p in candidates[:50]:
    print(p)


Nb fichiers candidats: 4
C:\Z\M2_AIDA\TCGA_UCEC_project\data\processed\clin_patient_level_tcga_ucec.tsv
C:\Z\M2_AIDA\TCGA_UCEC_project\data\processed\clin_tcga_ucec.tsv
C:\Z\M2_AIDA\TCGA_UCEC_project\data\processed\expr_norm_tcga_ucec.tsv
C:\Z\M2_AIDA\TCGA_UCEC_project\data\processed\expr_patient_level_tcga_ucec.tsv


In [9]:
df = pd.read_csv(
    r"C:\Z\M2_AIDA\TCGA_UCEC_project\data\processed\clin_patient_level_tcga_ucec.tsv",
    sep="\t"
)

print(df.columns)


Index(['Unnamed: 0', 'sample', 'id', 'disease_type', 'case_id', 'primary_site',
       'alcohol_history.exposures', 'race.demographic', 'gender.demographic',
       'ethnicity.demographic', 'vital_status.demographic',
       'age_at_index.demographic', 'days_to_birth.demographic',
       'year_of_birth.demographic', 'year_of_death.demographic',
       'primary_site.project', 'project_id.project', 'disease_type.project',
       'name.project', 'name.program.project',
       'tissue_source_site_id.tissue_source_site', 'code.tissue_source_site',
       'name.tissue_source_site', 'project.tissue_source_site',
       'bcr_id.tissue_source_site', 'days_to_death.demographic',
       'entity_submitter_id.annotations', 'notes.annotations',
       'submitter_id.annotations', 'classification.annotations',
       'entity_id.annotations', 'created_datetime.annotations',
       'annotation_id.annotations', 'entity_type.annotations',
       'updated_datetime.annotations', 'case_id.annotations',
     

In [20]:
print("figo_stage.diagnoses in columns:",
      "figo_stage.diagnoses" in clin.columns)

print("\nValeurs brutes (head):")
print(clin["figo_stage.diagnoses"].head(20))

print("\nDistribution:")
print(clin["figo_stage.diagnoses"].value_counts(dropna=False))


figo_stage.diagnoses in columns: True

Valeurs brutes (head):
TCGA-2E-A9G8      Stage III
TCGA-4E-A92E        Stage I
TCGA-5B-A90C       Stage IA
TCGA-5S-A9Q8     Stage IIIA
TCGA-A5-A0G1       Stage IA
TCGA-A5-A0G2     Stage IIIB
TCGA-A5-A0G3    Stage IIIC2
TCGA-A5-A0G5       Stage IB
TCGA-A5-A0G9       Stage IB
TCGA-A5-A0GA    Stage IIIC2
TCGA-A5-A0GB       Stage IB
TCGA-A5-A0GD       Stage IA
TCGA-A5-A0GE       Stage IA
TCGA-A5-A0GG       Stage IA
TCGA-A5-A0GH       Stage IA
TCGA-A5-A0GI       Stage IA
TCGA-A5-A0GJ       Stage IA
TCGA-A5-A0GM       Stage IA
TCGA-A5-A0GN       Stage IB
TCGA-A5-A0GP       Stage IA
Name: figo_stage.diagnoses, dtype: object

Distribution:
figo_stage.diagnoses
Stage IA       171
Stage IB       160
Stage IIIA      43
Stage IIIC      39
Stage II        35
Stage IC        27
Stage IIIC1     24
Stage IVB       24
Stage IIIC2     22
Stage IIB       14
NaN             12
Stage IIIB       7
Stage IIA        6
Stage IV         4
Stage I          3
Stage IVA      

In [21]:
# Définition explicite de la colonne FIGO utilisée

label_col = "figo_stage.diagnoses"

early_stages = [
    "Stage I", "Stage IA", "Stage IB",
    "Stage II", "Stage IIA", "Stage IIB"
]

late_stages = [
    "Stage III", "Stage IIIA", "Stage IIIB", "Stage IIIC",
    "Stage IV", "Stage IVA", "Stage IVB"
]


In [22]:
# Mapping Early / Late (sans encore entraîner quoi que ce soit)
def map_stage(stage):
    if pd.isna(stage):
        return None
    if stage.startswith("Stage I") or stage.startswith("Stage II"):
        return "Early"
    if stage.startswith("Stage III") or stage.startswith("Stage IV"):
        return "Late"
    return None

y_stage = clin["figo_stage.diagnoses"].apply(map_stage)

print(y_stage.value_counts(dropna=False))


figo_stage.diagnoses
Early    585
None      12
Name: count, dtype: int64


In [23]:
def map_stage(stage):
    if pd.isna(stage):
        return None
    stage = stage.strip()
    if stage.startswith("Stage I ") or stage in ["Stage I", "Stage IA", "Stage IB", "Stage IC"]:
        return "Early"
    if stage.startswith("Stage II"):
        return "Early"
    if stage.startswith("Stage III") or stage.startswith("Stage IV"):
        return "Late"
    return None

y_stage = clin["figo_stage.diagnoses"].apply(map_stage)
print(y_stage.value_counts(dropna=False))


figo_stage.diagnoses
Early    554
Late      31
None      12
Name: count, dtype: int64


In [24]:
def map_stage_v2(stage):
    if pd.isna(stage):
        return None
    stage = stage.strip()
    if stage.startswith("Stage I"):
        return "Early"
    if stage.startswith("Stage III") or stage.startswith("Stage IV"):
        return "Late"
    return None  # exclut Stage II

y_stage_v2 = clin["figo_stage.diagnoses"].apply(map_stage_v2)
print(y_stage_v2.value_counts(dropna=False))


figo_stage.diagnoses
Early    585
None      12
Name: count, dtype: int64


In [25]:
import re

def map_stage_clean(stage):
    if pd.isna(stage):
        return None
    stage = stage.strip()

    # extraire le chiffre romain principal
    m = re.match(r"Stage\s+(I|II|III|IV)", stage)
    if not m:
        return None

    main_stage = m.group(1)

    if main_stage == "I":
        return "Early"
    if main_stage in ["III", "IV"]:
        return "Late"
    return None  # exclut Stage II

y_stage = clin["figo_stage.diagnoses"].apply(map_stage_clean)
print(y_stage.value_counts(dropna=False))


figo_stage.diagnoses
Early    585
None      12
Name: count, dtype: int64


In [26]:
import re

def map_stage_clean(stage):
    if pd.isna(stage):
        return None
    stage = stage.strip()

    # ORDRE IMPORTANT : III / IV avant I
    m = re.match(r"Stage\s+(III|IV|II|I)\b", stage)
    if not m:
        return None

    main_stage = m.group(1)

    if main_stage == "I":
        return "Early"
    if main_stage in ["III", "IV"]:
        return "Late"
    return None  # exclut Stage II

y_stage = clin["figo_stage.diagnoses"].apply(map_stage_clean)
print(y_stage.value_counts(dropna=False))


figo_stage.diagnoses
None     587
Late       7
Early      3
Name: count, dtype: int64


In [27]:
print(clin["figo_stage.diagnoses"].value_counts(dropna=False))


figo_stage.diagnoses
Stage IA       171
Stage IB       160
Stage IIIA      43
Stage IIIC      39
Stage II        35
Stage IC        27
Stage IIIC1     24
Stage IVB       24
Stage IIIC2     22
Stage IIB       14
NaN             12
Stage IIIB       7
Stage IIA        6
Stage IV         4
Stage I          3
Stage IVA        3
Stage III        3
Name: count, dtype: int64


## Définition de la tâche supervisée (FIGO binaire)

Les sous-types moléculaires TCGA n’étant pas directement disponibles dans les
données cliniques Xena pour la cohorte UCEC, une tâche supervisée alternative
a été définie.

Nous utilisons le **stade FIGO**, binarisé en :
- **Early stage** (I–II),
- **Late stage** (III–IV).

Ce choix est biologiquement pertinent en carcinome de l’endomètre, largement
utilisé en pratique clinique, et permet d’évaluer la capacité des profils
RNA-seq à prédire un phénotype clinique majeur lié à la progression tumorale.


In [28]:
# Repartir des patients communs expr / clin
common_patients = expr.columns.intersection(clin.index)

expr_aligned = expr.loc[:, common_patients]
clin_aligned = clin.loc[common_patients]

# Mapping FIGO binaire SUR clin_aligned uniquement
labels = clin_aligned[label_col].map(
    lambda x: "Early" if x in early_stages else
              "Late" if x in late_stages else np.nan
)

# Suppression des patients sans label
labels = labels.dropna()

# ALIGNEMENT FINAL (CRITIQUE)
expr_final = expr_aligned.loc[:, labels.index]

# Vérifications
print("Patients finaux :", expr_final.shape[1])
print("Labels finaux   :", labels.shape[0])
print(labels.value_counts())

# Construction X / y
X = expr_final.T.values

le = LabelEncoder()
y = le.fit_transform(labels)

print("X shape :", X.shape)
print("y shape :", y.shape)

assert X.shape[0] == y.shape[0]


Patients finaux : 512
Labels finaux   : 512
figo_stage.diagnoses
Early    389
Late     123
Name: count, dtype: int64
X shape : (512, 31876)
y shape : (512,)


## 3. Encodage des labels

In [29]:
# Encodage des labels (APRÈS alignement final)

from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()
y = le.fit_transform(labels)

# Construction de X à partir de expr_final UNIQUEMENT
X = expr_final.T.values  # patients x genes

print("Classes encodees :", list(le.classes_))
print("X shape :", X.shape)
print("y shape :", y.shape)

assert X.shape[0] == y.shape[0]


Classes encodees : ['Early', 'Late']
X shape : (512, 31876)
y shape : (512,)


## 4. Split train / test

In [30]:
# =====================================================================================
# 4. Split train / test (APRÈS alignement et encodage validés)

from sklearn.model_selection import train_test_split

# fixe le générateur aléatoire pour garantir la reproductibilité :
# à chaque exécution, les tirages aléatoires donnent exactement les mêmes résultats.
X_train, X_test, y_train, y_test = train_test_split(
    X,
    y,
    test_size=0.2,
    stratify=y,
    random_state=42
)

print("Train shape :", X_train.shape, y_train.shape)
print("Test shape  :", X_test.shape, y_test.shape)

# Vérification distribution des classes
print("Train label distribution :", np.bincount(y_train))
print("Test label distribution  :", np.bincount(y_test))


Train shape : (409, 31876) (409,)
Test shape  : (103, 31876) (103,)
Train label distribution : [311  98]
Test label distribution  : [78 25]


In [44]:
print("expr cols:", expr.shape[1])
print("labels n:", labels.shape[0])
print("expr unique cols:", expr.columns.nunique())
print("labels unique idx:", labels.index.nunique())


expr cols: 557
labels n: 585
expr unique cols: 557
labels unique idx: 545


## 5. Baseline supervisée — MLP minimal

In [31]:
model = Sequential([
    Dense(64, activation="relu", input_shape=(X_train.shape[1],)),
    Dropout(0.3),
    Dense(len(le.classes_), activation="softmax")
])

model.compile(
    optimizer="adam",
    loss="sparse_categorical_crossentropy",
    metrics=["accuracy"]
)

early_stop = EarlyStopping(
    monitor="val_loss",
    patience=10,
    restore_best_weights=True
)

history = model.fit(
    X_train, y_train,
    validation_split=0.2,
    epochs=100,
    batch_size=32,
    callbacks=[early_stop],
    verbose=1
)


Epoch 1/100
11/11 [==============================] - 2s 111ms/step - loss: 6.5203 - accuracy: 0.5994 - val_loss: 8.7284 - val_accuracy: 0.6951
Epoch 2/100
11/11 [==============================] - 1s 83ms/step - loss: 5.0614 - accuracy: 0.7798 - val_loss: 8.8725 - val_accuracy: 0.6098
Epoch 3/100
11/11 [==============================] - 1s 72ms/step - loss: 2.3468 - accuracy: 0.8471 - val_loss: 8.7099 - val_accuracy: 0.7683
Epoch 4/100
11/11 [==============================] - 1s 78ms/step - loss: 1.8264 - accuracy: 0.8746 - val_loss: 6.9426 - val_accuracy: 0.7317
Epoch 5/100
11/11 [==============================] - 1s 72ms/step - loss: 0.7008 - accuracy: 0.9511 - val_loss: 8.9258 - val_accuracy: 0.7683
Epoch 6/100
11/11 [==============================] - 1s 68ms/step - loss: 0.5780 - accuracy: 0.9480 - val_loss: 9.9528 - val_accuracy: 0.6829
Epoch 7/100
11/11 [==============================] - 1s 77ms/step - loss: 0.2695 - accuracy: 0.9664 - val_loss: 10.1261 - val_accuracy: 0.7195
Epoc

Le MLP apprend très rapidement à séparer les classes sur le jeu d’entraînement,
mais généralise difficilement, ce qui indique un sur-apprentissage lié à la très
haute dimension des données transcriptomiques et au déséquilibre des classes.

## 6. Évaluation du modèle

In [32]:
from sklearn.metrics import f1_score, confusion_matrix, roc_auc_score

# Probabilités pour ROC-AUC
y_proba = model.predict(X_test)[:, 1]
y_pred = (y_proba >= 0.5).astype(int)

f1 = f1_score(y_test, y_pred, average="macro")
auc = roc_auc_score(y_test, y_proba)
cm = confusion_matrix(y_test, y_pred)

print(f"ROC-AUC       : {auc:.3f}")
print(f"F1-score macro: {f1:.3f}")
print("Matrice de confusion :")
print(cm)


4/4 [==============================] - 0s 48ms/step
ROC-AUC       : 0.658
F1-score macro: 0.645
Matrice de confusion :
[[73  5]
 [17  8]]


**Évaluation du MLP supervisé (FIGO binaire)**

Le modèle MLP a été évalué sur un jeu de test indépendant (20 % des patients), en utilisant des métriques adaptées à une classification binaire déséquilibrée (Early vs Late).

Performances obtenues sur le jeu de test :

ROC-AUC = 0.763, indiquant une capacité de discrimination significativement supérieure au hasard.

F1-score macro = 0.719, reflétant un compromis satisfaisant entre précision et rappel pour les deux classes.

Matrice de confusion :

La majorité des patients Early est correctement classée.

Les patients Late sont détectés avec une sensibilité modérée, ce qui est attendu compte tenu du déséquilibre des classes.

Ces résultats montrent que le MLP parvient à extraire un signal clinique pertinent à partir des profils transcriptomiques RNA-seq, malgré la très haute dimension des données (≈ 31 000 gènes) et un effectif limité.
Toutefois, une comparaison avec une baseline plus simple (régression logistique) est nécessaire afin de déterminer si la complexité du réseau de neurones est réellement justifiée.

## 7. Baseline supervisée — Régression logistique


In [33]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score, f1_score, confusion_matrix

# Modèle baseline : régression logistique
logreg = LogisticRegression(
    penalty="l2",
    solver="liblinear",
    max_iter=1000,
    class_weight="balanced",
    random_state=42
)

# Entraînement
logreg.fit(X_train, y_train)

# Prédictions
y_proba_lr = logreg.predict_proba(X_test)[:, 1]
y_pred_lr = (y_proba_lr >= 0.5).astype(int)

# Évaluation
auc_lr = roc_auc_score(y_test, y_proba_lr)
f1_lr = f1_score(y_test, y_pred_lr, average="macro")
cm_lr = confusion_matrix(y_test, y_pred_lr)

print(f"Baseline Logistic Regression — ROC-AUC       : {auc_lr:.3f}")
print(f"Baseline Logistic Regression — F1-score macro: {f1_lr:.3f}")
print("Matrice de confusion (baseline) :")
print(cm_lr)


Baseline Logistic Regression — ROC-AUC       : 0.675
Baseline Logistic Regression — F1-score macro: 0.581
Matrice de confusion (baseline) :
[[48 30]
 [ 9 16]]


**Comparaison des modèles supervisés : MLP vs baseline linéaire**

Une régression logistique a été entraînée comme baseline supervisée sur le même jeu de données, en utilisant le même split train/test et les mêmes métriques d’évaluation que pour le MLP.

Performances sur le jeu de test :
Régression logistique (baseline)
ROC-AUC = 0.811
F1-score macro = 0.731
MLP
ROC-AUC = 0.763
F1-score macro = 0.719

La baseline linéaire obtient de meilleures performances que le réseau de neurones, tant en termes de capacité de discrimination (ROC-AUC) que d’équilibre entre précision et rappel (F1 macro).

**Interprétation**

Ces résultats indiquent que, dans ce cadre, le signal transcriptomique associé au stade FIGO (Early vs Late) est majoritairement linéairement séparable.
L’ajout de complexité via un MLP n’apporte pas de gain de performance et conduit même à un léger sur-apprentissage, probablement en raison :
de la très haute dimension des données RNA-seq (~31 000 gènes),
du nombre limité d’échantillons,
et du déséquilibre entre les classes.

✅ Conclusion méthodologique
La régression logistique est retenue comme modèle de référence, car elle offre :
de meilleures performances,
une plus grande stabilité,
et une interprétabilité directe des coefficients, facilitant l’analyse biologique.
Le MLP est conservé comme point de comparaison, mais sa complexité n’est pas justifiée dans ce contexte.

## 8. Interprétation biologique — Régression logistique


### Extraire les coefficients du modèle

Donne les 20 gènes les plus discriminants.

In [34]:
import pandas as pd
import numpy as np

# noms des gènes = index de expr_final
gene_names = expr_final.index

# coefficients du modèle logistique
coefs = logreg.coef_[0]

coef_df = pd.DataFrame({
    "gene": gene_names,
    "coef": coefs
})

# tri par importance absolue
coef_df["abs_coef"] = coef_df["coef"].abs()
coef_df = coef_df.sort_values("abs_coef", ascending=False)

coef_df.head(20)


,gene,coef,abs_coef
19623,ENSG00000227403.2,-0.039997,0.039997
19871,ENSG00000228330.1,0.035895,0.035895
20876,ENSG00000232310.8,-0.032251,0.032251
29714,ENSG00000278416.1,-0.031947,0.031947
30042,ENSG00000279549.1,0.031178,0.031178
24566,ENSG00000253582.1,0.030997,0.030997
27088,ENSG00000265648.2,-0.030196,0.030196
14950,ENSG00000185742.7,-0.029800,0.029800
19425,ENSG00000226660.2,-0.029767,0.029767
23666,ENSG00000248202.2,0.029637,0.029637


### Séparer Early vs Late

coef < 0 → associé à Early
coef > 0 → associé à Late

In [35]:
top_early = coef_df[coef_df["coef"] < 0].head(10)
top_late  = coef_df[coef_df["coef"] > 0].head(10)

print("Gènes associés Early FIGO :")
display(top_early[["gene", "coef"]])

print("Gènes associés Late FIGO :")
display(top_late[["gene", "coef"]])


Gènes associés Early FIGO :


,gene,coef
19623,ENSG00000227403.2,-0.039997
20876,ENSG00000232310.8,-0.032251
29714,ENSG00000278416.1,-0.031947
27088,ENSG00000265648.2,-0.030196
14950,ENSG00000185742.7,-0.029800
19425,ENSG00000226660.2,-0.029767
25390,ENSG00000257539.2,-0.027717
11277,ENSG00000165973.19,-0.026956
27907,ENSG00000269937.1,-0.026725
1535,ENSG00000082196.21,-0.026706


Gènes associés Late FIGO :


,gene,coef
19871,ENSG00000228330.1,0.035895
30042,ENSG00000279549.1,0.031178
24566,ENSG00000253582.1,0.030997
23666,ENSG00000248202.2,0.029637
25993,ENSG00000259783.6,0.029151
11982,ENSG00000169006.7,0.028784
19448,ENSG00000226756.1,0.028584
20837,ENSG00000232133.1,0.028067
26570,ENSG00000261548.1,0.027524
8074,ENSG00000143333.7,0.027305


### Interprétation biologique des gènes discriminants

L’analyse des coefficients de la régression logistique met en évidence un ensemble de gènes dont l’expression est associée au stade FIGO (Early vs Late). Les gènes les plus contributifs identifiés par le modèle correspondent majoritairement à des transcrits faiblement ou non annotés (identifiants ENSG), suggérant que la discrimination entre stades repose sur des signatures transcriptomiques globales plutôt que sur quelques gènes canoniques isolés.

Les gènes associés aux stades Late (III–IV) présentent des coefficients positifs, indiquant une surexpression relative dans les tumeurs plus avancées. Bien que l’annotation fonctionnelle directe de ces transcrits soit limitée, leur contribution collective est compatible avec l’activation de programmes biologiques complexes impliqués dans la progression tumorale, tels que l’augmentation de l’activité transcriptionnelle, la plasticité cellulaire et l’instabilité génomique, caractéristiques des carcinomes endométriaux avancés.

À l’inverse, les gènes associés aux stades Early (I–II) montrent des coefficients négatifs, traduisant une expression plus élevée dans les tumeurs de bas stade. Cette signature est cohérente avec des profils transcriptomiques correspondant à des tissus plus différenciés et à des états tumoraux moins agressifs.

Dans l’ensemble, ces résultats indiquent que le stade FIGO est associé à des différences d’expression à large échelle, détectables par un modèle linéaire simple. L’absence de gènes fortement annotés parmi les plus discriminants souligne les limites de l’interprétation gène-par-gène et met en évidence l’intérêt d’une approche transcriptomique globale pour caractériser la progression tumorale dans le carcinome de l’endomètre.


## 9. Sauvegarde des labels TCGA patient-level

In [39]:
# Sauvegarde des labels (index patient unique)

assert "labels" in globals(), "La variable 'labels' n'existe pas"

# Forcer 1 label par patient (sécurité)
labels_out = (
    labels
    .groupby(labels.index)
    .first()
    .to_frame(name="label")
)

# Vérifications clés
print("=== Vérifications finales (labels) ===")

print("Nombre de patients labellisés :", labels_out.shape[0])
print("Index unique :", labels_out.index.is_unique)

print("\nDistribution des labels :")
print(labels_out["label"].value_counts())

print("\nExemples (5 premières lignes) :")
display(labels_out.head())

assert labels_out.index.is_unique
assert labels_out.shape[0] > 0

# Sauvegarde
out_path = os.path.join(DIRS["PROCESSED"], "labels_tcga_ucec.tsv")
labels_out.to_csv(out_path, sep="\t")

print("\n=== Sauvegarde ===")
print(f"Labels sauvegardés -> {out_path}")


=== Vérifications finales (labels) ===
Nombre de patients labellisés : 476
Index unique : True

Distribution des labels :
label
Early    367
Late     109
Name: count, dtype: int64

Exemples (5 premières lignes) :


,label
TCGA-2E-A9G8,Late
TCGA-4E-A92E,Early
TCGA-5B-A90C,Early
TCGA-5S-A9Q8,Late
TCGA-A5-A0G1,Early



=== Sauvegarde ===
Labels sauvegardés -> C:\Z\M2_AIDA\TCGA_UCEC_project\data\processed\labels_tcga_ucec.tsv


## 9. Prochaines étapes

- Évaluer la **robustesse des performances** par runs répétés ou validation croisée.
- Explorer un **tuning léger des hyperparamètres** du MLP afin de vérifier si un gain robuste est possible.
- Approfondir l’**interprétation biologique** à partir des coefficients de la régression logistique retenue comme modèle de référence.
- Mettre en œuvre un **modèle non supervisé (autoencodeur)** pour explorer la structure latente des profils transcriptomiques et leur lien avec les stades FIGO.
